# **Fiber photometry of SNr GABAergic neurons during optogenetic stimulation of STN and PPN inputs in freely moving mice**

This tutorial shows how to access and process data from [DANDI:001528](https://dandiarchive.org/dandiset/001528/draft) for the study detailed in [*"Parkinson's Disease-vulnerable and -resilient dopamine neurons display opposite responses to excitatory input"*](https://www.biorxiv.org/content/10.1101/2025.06.03.657460v1)

## Study Overview

This dataset contains fiber photometry recordings from substantia nigra (SN) GABAergic neurons in freely moving mice during:
- **Day 1**: Varying duration optogenetic stimulation (250ms, 1s, 4s at 40Hz)
- **Day 2**: Varying frequency optogenetic stimulation (5Hz, 10Hz, 20Hz, 40Hz)
- **Day 3**: Electrical shock delivery (varying durations)

## Contents

1. [Setup and Data Access](#setup)
2. [Session and Subject Metadata](#metadata)
3. [Fiber Photometry Data and Metadata](#photometry)
4. [Behavioral Video](#video)
5. [Session Type 1: Varying Duration Optogenetic Stimulation](#session1)
6. [Session Type 2: Varying Frequency Optogenetic Stimulation](#session2)
7. [Session Type 3: Electrical Shock Delivery](#session3)

---

# 1. Setup and Data Access <a id="setup"></a>

## Import Required Libraries

In [ ]:
# Core data manipulation and analysis
from pathlib import Path

import h5py

# Visualization
import matplotlib.pyplot as plt
import numpy as np
import remfile
from dandi.dandiapi import DandiAPIClient

# NWB and DANDI access
from pynwb import NWBHDF5IO

# Configure matplotlib
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

## Data Access Functions

In [ ]:
def load_nwb_from_dandi(dandiset_id, subject_id, session_id):
    """
    Load NWB file from DANDI Archive via streaming.
    """
    pattern = f"sub-{subject_id}/sub-{subject_id}_ses-{session_id}*.nwb"
    
    with DandiAPIClient() as client:
        client.dandi_authenticate()
        assets = client.get_dandiset(dandiset_id, "draft").get_assets_by_glob(
            pattern=pattern, order="path"
        )
        
        s3_urls = []
        for asset in assets:
            s3_url = asset.get_content_url(follow_redirects=1, strip_query=False)
            s3_urls.append(s3_url)
        
        if len(s3_urls) != 1:
            raise ValueError(f"Expected 1 file, found {len(s3_urls)} for pattern {pattern}")
        
        s3_url = s3_urls[0]
    
    file = remfile.File(s3_url)
    h5_file = h5py.File(file, "r")
    io = NWBHDF5IO(file=h5_file, load_namespaces=True)
    nwbfile = io.read()
    
    return nwbfile, io


def load_nwb_local(directory_path, subject_id, session_id):
    """
    Load NWB file from local directory.
    """
    directory_path = Path(directory_path)
    nwbfile_path = directory_path / f"sub-{subject_id}_ses-{session_id}.nwb"
    
    if not nwbfile_path.exists():
        raise FileNotFoundError(f"NWB file not found: {nwbfile_path}")
    
    io = NWBHDF5IO(path=nwbfile_path, load_namespaces=True)
    nwbfile = io.read()
    
    return nwbfile, io

---

# 2. Session and Subject Metadata <a id="metadata"></a>

In [ ]:
# Load session data
dandiset_id = "001528"
subject_id = "C4561"  # Example subject
session_id = "varying-duration"  # Start with first session type

# Choose data source (DANDI streaming or local)
USE_DANDI = True  # Set to False to use local files

if USE_DANDI:
    nwbfile, io = load_nwb_from_dandi(dandiset_id, subject_id, session_id)
else:
    # Specify your local directory path
    local_directory = "YOUR_DIRECTORY_PATH"  # Replace with actual path
    nwbfile, io = load_nwb_local(local_directory, subject_id, session_id)

print("=== SESSION INFORMATION ===")
print(f"Session description: {nwbfile.session_description}")
print(f"Session start time: {nwbfile.session_start_time}")
print(f"Experiment description: {nwbfile.experiment_description}")

In [ ]:
print("\n=== SUBJECT INFORMATION ===")
nwbfile.subject

---

# 3. Fiber Photometry Data and Metadata <a id="photometry"></a>

In [ ]:
# Access raw fiber photometry data
raw_signal = nwbfile.acquisition["raw_modulated_signal"]

print("=== RAW FIBER PHOTOMETRY SIGNAL ===")
print(f"Description: {raw_signal.description}")
print(f"Data shape: {raw_signal.data.shape}")
print(f"Sampling rate: {raw_signal.rate} Hz")
print(f"Duration: {raw_signal.data.shape[0] / raw_signal.rate:.2f} seconds")

In [ ]:
# Plot raw acquisition signal (simple view)
data = raw_signal.data[:]
timestamps = raw_signal.get_timestamps()

# Show first 30 seconds
end_idx = int(30 * raw_signal.rate)
time_subset = timestamps[:end_idx]
data_subset = data[:end_idx]

plt.figure(figsize=(15, 2))
plt.plot(time_subset, data_subset, linewidth=0.5)
plt.xlabel('Time (s)')
plt.ylabel('Fluorescence (a.u.)')
plt.title('Raw Modulated Fiber Photometry Signal (first 30 seconds)')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Access fiber photometry metadata
fiber_photometry_table = nwbfile.lab_meta_data["fiber_photometry"].fiber_photometry_table[:]
print("=== FIBER PHOTOMETRY METADATA ===")
print(f"Optical fiber: {fiber_photometry_table['optical_fiber'][0]}")
print(f"Indicator: {fiber_photometry_table['indicator'][0]}")
print(f"Excitation source: {fiber_photometry_table['excitation_source'][0]}")
print(f"Photodetector: {fiber_photometry_table['photodetector'][0]}")

---

# 4. Behavioral Video <a id="video"></a>

In [ ]:
# Access behavioral video information
if "BehavioralVideo" in nwbfile.acquisition:
    video = nwbfile.acquisition["BehavioralVideo"]

    print("=== BEHAVIORAL VIDEO ===")
    print(f"Video file path: {video.external_file[0]}")
    print(f"Description: {video.description}")
    print(f"Video start: {video.timestamps[0]} seconds")
    print(f"Video end: {video.timestamps[-1]} seconds")
else:
    print("No behavioral video found in this session.")

In [ ]:
video.device  # Display video device information

---

# 5. Session Type 1: Varying Duration Optogenetic Stimulation <a id="session1"></a>

This session contains 40 Hz optogenetic stimulation at three different durations: 250ms, 1s, and 4s.

In [ ]:
# Access optogenetic stimulation data
optogenetic_intervals = nwbfile.stimulus["optogenetic_stimulus_interval"]
intervals_df = optogenetic_intervals.to_dataframe()

print("=== OPTOGENETIC STIMULATION INTERVALS ===")
print(f"Total number of stimuli: {len(intervals_df)}")
print("\nFirst 5 intervals:")
intervals_df.head()

In [ ]:
# Access demodulated signals
module = nwbfile.processing["ophys"]
calcium_signal = module["calcium_signal"]
isosbestic_signal = module["isosbestic_signal"]

# Get signal data
calcium_data = calcium_signal.data[:]
isosbestic_data = isosbestic_signal.data[:]
rate = calcium_signal.rate
start_time = calcium_signal.starting_time
timestamps = start_time + np.arange(len(calcium_data)) / rate

print(f"Demodulated calcium signal duration: {len(calcium_data) / rate:.2f} seconds")
print(f"Sampling rate: {rate:.2f} Hz")

Associated metadata

In [ ]:
calcium_signal.fiber_photometry_table_region[0]["excitation_source"][0]

In [ ]:
calcium_signal.fiber_photometry_table_region[0]["optical_fiber"][0]

In [ ]:
calcium_signal.fiber_photometry_table_region[0]["photodetector"][0]

In [ ]:
# Plot optogenetic stimulation and demodulated signal together
fig, ax = plt.subplots(1, 1, figsize=(14, 6))

# Plot demodulated calcium signal
ax.plot(timestamps, calcium_data, color="blue", linewidth=0.8, alpha=0.8, label="Calcium Signal")
ax.plot(timestamps, isosbestic_data, color="red", linewidth=0.8, alpha=0.6, label="Isosbestic Signal")
ax.set_xlabel('Time (s)')
ax.set_ylabel('Demodulated Signal (a.u.)')
ax.set_title('Demodulated Calcium Signal with Optogenetic Stimulation (40 Hz, Varying Duration)')
ax.grid(True, alpha=0.3)
ax.legend()

# Add stimulus intervals as faded boxes over the signal
duration_colors = {0.25: 'red', 1.0: 'blue', 4.0: 'green'}
duration_labels = {0.25: "250ms stimulus", 1.0: "1s stimulus", 4.0: "4s stimulus"}

for i, row in intervals_df.iterrows():
    duration = row['stop_time'] - row['start_time']
    # Round duration to nearest expected value for color mapping
    rounded_duration = round(duration * 4) / 4  # Round to nearest 0.25
    color = duration_colors.get(rounded_duration, 'gray')
    ax.axvspan(
        row["start_time"],
        row["stop_time"],
        color=color,
        alpha=0.2,
        label=duration_labels.get(rounded_duration, f"{rounded_duration}s"),
    )
    # Remove duplicate legend entries
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys())
plt.tight_layout()
plt.show()

---

# 6. Session Type 2: Varying Frequency Optogenetic Stimulation <a id="session2"></a>

This session contains optogenetic stimulation at varying frequencies (5Hz, 10Hz, 20Hz, 40Hz).

In [ ]:
# Load varying frequency session
session_id_freq = "varying-frequencies"

if USE_DANDI:
    nwbfile_freq, io_freq = load_nwb_from_dandi(dandiset_id, subject_id, session_id_freq)
else:
    nwbfile_freq, io_freq = load_nwb_local(local_directory, subject_id, session_id_freq)

print(f"Loaded session: {nwbfile_freq.session_description}")

### Access trial table

In [ ]:
trials_freq_df = nwbfile_freq.trials.to_dataframe()
print("=== TRIALS TABLE ===")
print(f"Total number of trials: {len(trials_freq_df)}")
trials_freq_df

### Access optogenetic stimulus

In [ ]:
# Access frequency session data
optogenetic_intervals_freq = nwbfile_freq.stimulus["optogenetic_stimulus_interval"]
intervals_freq_df = optogenetic_intervals_freq.to_dataframe()

print("=== VARYING FREQUENCY STIMULATION ===")
print(f"Total number of stimuli: {len(intervals_freq_df)}")
print("\nFirst 5 intervals:")
intervals_freq_df.sort_values(by='start_time', inplace=True)
intervals_freq_df.head()

### Access demodulated signal

In [ ]:
module_freq = nwbfile_freq.processing["ophys"]
calcium_signal_freq = module_freq["calcium_signal"]
calcium_data_freq = calcium_signal_freq.data[:]
isosbestic_data_freq = module_freq["isosbestic_signal"].data[:]
rate_freq = calcium_signal_freq.rate
start_time_freq = calcium_signal_freq.starting_time
timestamps_freq = start_time_freq + np.arange(len(calcium_data_freq)) / rate_freq

In [ ]:
# Plot optogenetic stimulation and demodulated signal together
fig, axs = plt.subplots(3, 1, figsize=(14, 6))

# Plot first trial
for i in range(3):
    trial_start_time = trials_freq_df.iloc[i]["start_time"]
    trial_end_time = trials_freq_df.iloc[i]["stop_time"]
    tag = trials_freq_df.iloc[i]["tags"]
    timestamps = timestamps_freq[(timestamps_freq >= trial_start_time) & (timestamps_freq <= trial_end_time)]
    calcium_data = calcium_data_freq[(timestamps_freq >= trial_start_time) & (timestamps_freq <= trial_end_time)]
    isosbestic_data = isosbestic_data_freq[(timestamps_freq >= trial_start_time) & (timestamps_freq <= trial_end_time)]
    # Plot demodulated calcium signal
    axs[i].plot(timestamps, calcium_data, color="blue", linewidth=0.8, alpha=0.8, label="Calcium Signal")
    axs[i].plot(timestamps, isosbestic_data, color="red", linewidth=0.8, alpha=0.6, label="Isosbestic Signal")
    axs[i].set_xlabel("Time (s)")
    axs[i].set_ylabel("Demodulated Signal (a.u.)")
    axs[i].set_title(f"Demodulated Calcium Signal with Optogenetic Stimulation ({tag}, Varying Frequencies)")
    axs[i].grid(True, alpha=0.3)

    # Add stimulus intervals as faded boxes over the signal
    frequency_colors = {5.0: "red", 10.0: "blue", 20.0: "green", 40.0: "yellow"}
    frequency_labels = {5.0: "5Hz stimulus", 10.0: "10Hz stimulus", 20.0: "20Hz stimulus", 40.0: "40Hz stimulus"}

    for r, row in intervals_freq_df.iterrows():
        # select rows within the trial time
        if not (row["start_time"] >= trial_start_time and row["stop_time"] <= trial_end_time):
            continue
        frequency = row["stimulus_frequency"]
        color = frequency_colors.get(frequency, "gray")
        axs[i].axvspan(
            row["start_time"],
            row["stop_time"],
            color=color,
            alpha=0.2,
            label=frequency_labels.get(frequency, f"{frequency}Hz"),
        )

    handles, labels = axs[i].get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    axs[0].legend(by_label.values(), by_label.keys())

plt.tight_layout()
plt.show()

---

# 7. Session Type 3: Electrical Shock Delivery <a id="session3"></a>

This session contains uncued electrical shocks at varying durations.

In [ ]:
# Load shock session
session_id_shock = "shock"

if USE_DANDI:
    nwbfile_shock, io_shock = load_nwb_from_dandi(dandiset_id, subject_id, session_id_shock)
else:
    nwbfile_shock, io_shock = load_nwb_local(local_directory, subject_id, session_id_shock)

print(f"Loaded session: {nwbfile_shock.session_description}")

In [ ]:
intervals_shock_df = nwbfile_shock.stimulus["auditory_stimulus_interval"].to_dataframe()
intervals_shock_df.head()

In [ ]:
# Access demodulated signal for shock session
module_shock = nwbfile_shock.processing["ophys"]
calcium_signal_shock = module_shock["calcium_signal"]
calcium_data_shock = calcium_signal_shock.data[:]
isosbestic_data_shock = module_shock["isosbestic_signal"].data[:]
rate_shock = calcium_signal_shock.rate
start_time_shock = calcium_signal_shock.starting_time
timestamps_shock = start_time_shock + np.arange(len(calcium_data_shock)) / rate_shock

print(f"Shock session signal duration: {len(calcium_data_shock) / rate_shock:.2f} seconds")

In [ ]:
# Plot auditory stimulation and demodulated signal together
fig, ax = plt.subplots(1, 1, figsize=(14, 6))

# Plot demodulated calcium signal
ax.plot(timestamps_shock, calcium_data_shock, color="blue", linewidth=0.8, alpha=0.8, label="Calcium Signal")
ax.plot(timestamps_shock, isosbestic_data_shock, color="red", linewidth=0.8, alpha=0.6, label="Isosbestic Signal")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Demodulated Signal (a.u.)")
ax.set_title("Demodulated Calcium Signal with Auditory Stimulation (Paried or not with shocks)")
ax.grid(True, alpha=0.3)
ax.legend()

# Add stimulus intervals as faded boxes over the signal
shock_colors = {"shock": "red", "no shock": "blue"}
shock_labels = {"shock": "paired with shock", "no shock": "not paired with shock"}

for i, row in intervals_shock_df.iterrows():
    shock = row["paired_shock_stimulus"]
    color = shock_colors.get(shock, "gray")
    ax.axvspan(
        row["start_time"],
        row["stop_time"],
        color=color,
        alpha=0.2,
        label=shock_labels.get(shock, f"{shock}"),
    )
    # Remove duplicate legend entries
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys())
plt.tight_layout()
plt.show()

In [ ]:
# Clean up - close IO objects
if 'io' in locals():
    io.close()
if 'io_freq' in locals():
    io_freq.close()
if 'io_shock' in locals():
    io_shock.close()
    
print("Tutorial completed successfully!")
print("All IO objects have been closed.")